In [76]:
import tensorflow as tf
import numpy as np
import csv
import pickle
import os
import ast
from glob import iglob

### VOCAB파일 불러오기

In [63]:
vocab_file = './vocab.csv'
with open(vocab_file, 'r') as f:
    words = csv.reader(f)
    vocabs = []
    for i, word in enumerate(words):
        vocabs.append(word[0])
word_to_id = {w:i for i,w in enumerate(vocabs)}
len(vocabs), type(vocabs), len(word_to_id), type(word_to_id)

(13841, list, 13841, dict)

### 데이터 불러오기

In [31]:
data_samples = 'data_samples'
subdirectories = [os.path.join(data_samples,o) for o in os.listdir(data_samples) if os.path.isdir(os.path.join(data_samples,o))]
python_files = [(directory, [y for x in os.walk(directory) for y in iglob(os.path.join(x[0], '*.py'))])
                for directory in subdirectories]

In [55]:
folder_name = python_files[0][0]
file_names = python_files[0][1]

In [73]:
mapping = (lambda x: x) if word_to_id is None else (lambda x: word_to_id.get(x, oov_id))
def get_source_tree(filename):
    print(filename)
    with open(filename, 'r', encoding='utf-8') as f:
        fstr = f.read()
    fstr = fstr.replace('\\n', '\n').replace('\r', '\n')
    if not fstr.endswith('\n'):
        fstr += '\n'
    return fstr, ast.parse(fstr, filename=filename)

def preprocess(tokentype, tokenval):
    if tokentype == tokenize.NUMBER:
        return number_token

    elif tokentype == tokenize.INDENT:
        return indent_token

    elif tokentype == tokenize.DEDENT:
        return dedent_token

    return tokenval

In [83]:
data = []
definition_positions = []
identifier_usage = []
gen_def_positions = True
error_data = 0
data_file_name = []
for filename in file_names:
#     try:
        source, tree = get_source_tree(filename)
        tokens = tokenize.generate_tokens(StringIO(source).readline)
        data.append([(mapping(preprocess(tokenType, tokenVal)), start) for tokenType, tokenVal, start, _, _
                     in tokens
                     if tokenType != tokenize.COMMENT and
                     not tokenVal.startswith("'''") and
                     not tokenVal.startswith('"""') and 
                     (tokenType == tokenize.DEDENT or tokenVal != "")])
        data_file_name.append(filename)
        if gen_def_positions:
            walker = astwalker.ASTWalker()
            walker.walk(tree)
            definition_positions.append(walker.definition_positions)
            identifier_usage.append(walker.name_usage)
#     except:
#         error_data+=1
#         continue


data_samples\aio-libs\aiohttp\setup.py


SyntaxError: EOL while scanning string literal (setup.py, line 59)

In [82]:
data

[]